In [ ]:
!pip install transformers datasets torch
!pip install peft==0.3.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.13.2
    Unin

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset

# Load the T5-small model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Load your dataset (replace with your own dataset or use a Hugging Face dataset)
dataset = load_dataset("scientific_papers", "arxiv", split="train[:1%]")  # Use a subset for quick testing


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

...

scientific_papers.py:   0%|          | 0.00/5.35k [00:00<?, ?B/s]

The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [ ]:
def preprocess_data(example):
    # Concatenate 'summarize: ' prefix to the input text
    input_text = "summarize: " + example['article']
    target_text = example['abstract']  # target summary text

    # Tokenize inputs and labels
    inputs = tokenizer(input_text, max_length=1024, truncation=True, padding="max_length")
    targets = tokenizer(target_text, max_length=256, truncation=True, padding="max_length")

    inputs['labels'] = targets['input_ids']
    return inputs

# Apply the preprocessing function to the dataset
tokenized_dataset = dataset.map(preprocess_data, remove_columns=["article", "abstract"])

Map:   0%|          | 0/2030 [00:00<?, ? examples/s]

In [ ]:
# Install the necessary library for PEFT/LoRA if not installed
!pip install peft

from transformers import Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig  # Import PEFT library for LoRA
import os

# Disable wandb if you don’t need logging
os.environ["WANDB_MODE"] = "disabled"

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,  # rank
    lora_alpha=32,  # scaling factor
    lora_dropout=0.1,  # dropout for LoRA layers
    bias="none"  # use "none" for no additional bias parameters
)

# Apply LoRA to your model
model = get_peft_model(model, lora_config)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",           # Evaluate only at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,   # Increased batch size
    num_train_epochs=2,              # Reduced number of epochs
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,                # Reduced logging frequency
    fp16=True,                       # Mixed precision for faster training
)
# Use the preprocessed dataset (tokenized_dataset) instead of the original dataset
train_dataset = tokenized_dataset.train_test_split(test_size=0.1)['train']
eval_dataset = tokenized_dataset.train_test_split(test_size=0.1)['test']

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset  # Pass the eval_dataset
)

# Start training
trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,6.274800,No log
2,4.854300,No log


TrainOutput(global_step=458, training_loss=6.0769510977132875, metrics={'train_runtime': 225.3316, 'train_samples_per_second': 16.216, 'train_steps_per_second': 2.033, 'total_flos': 995698711461888.0, 'train_loss': 6.0769510977132875, 'epoch': 2.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model.save_pretrained("./fine_tuned_t5_summarizer")
tokenizer.save_pretrained("./fine_tuned_t5_tockenizer")
!cp -r ./fine_tuned_t5_summarizer /content/drive/MyDrive/
!cp -r ./fine_tuned_t5_tockenizer /content/drive/MyDrive/


Mounted at /content/drive


In [ ]:
import os
os.getcwd()

In [ ]:
save_path = '/content/drive/My Drive/fine_tuned_t5_summarizer'

# Save the fine-tuned model
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

In [ ]:
!ls

sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')